# Preprocess static data from DB Timetables API

- request departures and arrivals from / to relevant stations for the whole day
- filter trains for relevant service types (e.g. ICE / S / RE etc.)
- save to disk

runs once at startup and once every night at 0:05 am

## Request timetables for the current day from 1-23 am (0am doesnt work for some reason)

### helper functions for handling db dates

In [59]:
import datetime
from dotenv import load_dotenv
from os import getenv

load_dotenv()



True

In [60]:
# download all 23 available timeslots for the day
import requests
import xmltodict

# load envs
db_timetables_base_url = getenv("db_timetables_base_url")
db_client_id = getenv("db_client_id")
db_client_secret = getenv("db_client_secret")

eva_number_heidelberg_hbf = 8000156

# prepare date
date_today = datetime.date.today()
dbdate_today = dateToDBDate(date_today)

print(dbdate_today)

# prepare request headers
auth_headers = {
    'DB-Client-Id': db_client_id,
    'DB-Api-Key': db_client_secret
}

responses_xml = []

for hour in range(1,24):
    # {hour:02} means print hour, fill (0) up to 2 (2) digits
    url = f"{db_timetables_base_url}/plan/{eva_number_heidelberg_hbf}/{dbdate_today}/{hour:02}"
    
    response = requests.get(url=url, headers=auth_headers)
    if response.status_code != 200:
        print(f"skipping hour {hour}, {response.status_code}")
        continue
    responses_xml.append(response)
    

250810
skipping hour 1, 404
skipping hour 2, 404
skipping hour 3, 404


## process xml responses

now that we have saved all the xmls we need, we can go through them one by one, extract the relevant attributes and save them in a dataframe 

In [ ]:
import pandas as pd

timetable_rows = []

for response_xml in responses_xml:
    # convert response_xml to dict
    timetable = xmltodict.parse(response_xml.content)

    timetable_stops = timetable['timetable']['s']
    
    for timetable_stop in timetable_stops:
        # category (e.g. ICE, RE, S)
        trip_label = timetable_stop['tl']
        category = trip_label['@c']
        
        
        # arrival
        arrival_dbtime = None
        arrival_path = None
        
        if 'ar' in timetable_stop:
            arrival = timetable_stop['ar']
            arrival_dbtime = arrival['@pt']
            arrival_path = arrival['@ppth']
            
        # departure
        departure_dbtime = None
        departure_path = None
        
        if 'dp' in timetable_stop:
            departure = timetable_stop['dp']
            departure_dbtime = departure['@pt']
            departure_path = departure['@ppth']
            
            
        
        # line (just for development)
        line = None
        
        if category in ["ICE", "IC", "FLX", "RJ", "RJX", "NJ", "TGV"]:
            line = trip_label['@n']
        elif 'ar' in timetable_stop:
            line = timetable_stop['ar']['@l']
        elif 'dp' in timetable_stop:
            line = timetable_stop['dp']['@l']
            
        
        
        timetable_row = pd.DataFrame(data={'category':[category], 'line': [line], 'arrival_dbtime': [arrival_dbtime], 'arrival_path':[arrival_path], 'departure_dbtime':[departure_dbtime], 'departure_path':[departure_path]})
        timetable_rows.append(timetable_row)

df_timetable = pd.concat(timetable_rows, ignore_index=True)
print(df_timetable.head(5))



  category line arrival_dbtime  \
0        S    3     2508100453   
1        S    2     2508100429   
2        S    2           None   
3        S    1     2508100403   
4       RB   68           None   

                                        arrival_path departure_dbtime  \
0  Ludwigshafen(Rh)Hbf|Ludwigshafen(Rhein) Mitte|...       2508100455   
1  Mannheim Hbf|Mannheim-Friedrichsfeld Süd|Heide...             None   
2                                               None       2508100432   
3  Karlsruhe Hbf|Karlsruhe-Durlach|Weingarten(Bad...       2508100413   
4                                               None       2508100520   

                                      departure_path  
0  Heidelberg-Kirchheim/Rohrbach|St Ilgen-Sandhau...  
1                                               None  
2  Heidelberg-Pfaffengrund/Wieblingen|Mannheim-Fr...  
3  Heidelberg-Pfaffengrund/Wieblingen|Mannheim-Fr...  
4  Heidelberg-Pfaffengrund/Wieblingen|Neu-Edingen...  


finally, save the timetable to disk for later use throughout the day

In [62]:
df_timetable.to_csv('db_timetable.csv', index=False)